In [2]:
!pip install transformers torch datasets

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/ec/93/454ada0d1b289a0f4a86ac88dbdeab54921becabac45da3da787d136628f/datasets-2.16.1-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow-hotfix from https://files.pythonhosted.org/packages/e4/f4/9ec2222f5f5f8ea04f66f184caafd991a39c8782e31f5b0266f101cb68ca/pyarrow_hotfix-0.6-py3-none-any.whl.metadata
  Obtaining dependency information for xxhash from https://files.pythonhosted.org/packages/b7/3a/74a609706ef4430fe6d041a3b8d209882c15440b695e373fe26d48c6f35c/xxhash-3.4.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for multiprocess from https://files.pythonhosted.org/packages/e7/41/96ac938770ba6e7d5ae1d8c9cafebac54b413549042c6260f0d0a6ec6622/multiprocess-0.70.15-py311-none-any.whl.metadata
  Obtaining dependency information for fsspec[http]<=2023.10.0,>=2023.1.0 from https://fil

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2023.3.0 requires fsspec==2023.3.0, but you have fsspec 2023.10.0 which is incompatible.


# Loading Datasets From Hugging Face

In [3]:
from datasets import load_dataset

dataset = load_dataset("fka/awesome-chatgpt-prompts")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['act', 'prompt'],
        num_rows: 153
    })
})

In [4]:
!pip install py7zr

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for py7zr from https://files.pythonhosted.org/packages/c0/39/71263be4c6081e1788f4da35e937fca1808914b0d98a04bd44255b948699/py7zr-0.20.8-py3-none-any.whl.metadata
  Obtaining dependency information for texttable from https://files.pythonhosted.org/packages/24/99/4772b8e00a136f3e01236de33b0efda31ee7077203ba5967fcc76da94d65/texttable-1.7.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pycryptodomex>=3.16.0 from https://files.pythonhosted.org/packages/b2/e8/1b92184ab7e5595bf38000587e6f8cf9556ebd1bf0a583619bee2057afbd/pycryptodomex-3.20.0-cp35-abi3-win_amd64.whl.metadata
  Obtaining dependency information for pyzstd>=0.15.9 from https://files.pythonhosted.org/packages/c0/ac/7d30ccaec1479ce0dfc33494ed0d856ad7655977891c79fbef317ade9835/pyzstd-0.15.9-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for pyppmd<1.2.0,>=1.1.0 from https://file

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [5]:
from datasets import load_dataset

dataset = load_dataset("samsum")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

# Data Preprocessing Methods

In [6]:
# Reload GPT Prompt dataset (Cache keeps from loading it twice)
dataset = load_dataset("fka/awesome-chatgpt-prompts")
dataset

DatasetDict({
    train: Dataset({
        features: ['act', 'prompt'],
        num_rows: 153
    })
})

In [7]:
# print an example

dataset['train'][0]

{'act': 'Linux Terminal',
 'prompt': 'I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is pwd'}

In [8]:
# shuffle & sample

dataset = dataset['train'].shuffle(seed=37).select(range(100))
dataset

Dataset({
    features: ['act', 'prompt'],
    num_rows: 100
})

In [9]:
# Create Test Dataset

dataset = dataset.train_test_split(train_size=0.8, seed=42)
dataset

DatasetDict({
    train: Dataset({
        features: ['act', 'prompt'],
        num_rows: 80
    })
    test: Dataset({
        features: ['act', 'prompt'],
        num_rows: 20
    })
})

# Creating Your Own Dataset

In [10]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/reuters21578-mld/reuters21578.tar.gz

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
!tar -xzvf reuters21578.tar.gz

tar: Error opening archive: Failed to open 'reuters21578.tar.gz'


In [12]:
# the sgm files are what contains the articles
from bs4 import BeautifulSoup

# Open the file and parse its content with BeautifulSoup
reuters_articles = []
for i in range(22):
  if i < 10:
    i = f"0{i}"

  # load file data
  with open(f"/content/reut2-0{i}.sgm", 'r', encoding='latin-1') as file:
      soup = BeautifulSoup(file, "html.parser")

  # Extract articles' titles and bodies
  articles = []
  for reuters in soup.find_all('reuters'):
      title = reuters.title.string if reuters.title else ""
      body = reuters.body.string if reuters.body else ""
      articles.append({
            'title': title,
            'body': body
        })

  reuters_articles.extend(articles)

FileNotFoundError: [Errno 2] No such file or directory: '/content/reut2-000.sgm'

In [ ]:
# Print out the first few articles for inspection
for i, article in enumerate(reuters_articles[:5]):
  print(article)
  print("-"*100)

In [ ]:
len(reuters_articles)

In [ ]:
import json

TRAIN_PCT, VALID_PCT = 0.8, 0.1

# Split the data
train_articles = reuters_articles[:int(len(reuters_articles)*TRAIN_PCT)]
valid_articles = reuters_articles[int(len(reuters_articles)*TRAIN_PCT): int(len(reuters_articles)*(TRAIN_PCT + VALID_PCT))]
test_articles = reuters_articles[int(len(reuters_articles)*(TRAIN_PCT + VALID_PCT)):]

# Function to save articles as JSONL
def save_as_jsonl(data, filename):
    with open(filename, "w") as f:
        for article in data:
            f.write(json.dumps(article) + "\n")

# Save them into temporary JSONL files
save_as_jsonl(train_articles, "train.jsonl")
save_as_jsonl(valid_articles, "valid.jsonl")
save_as_jsonl(test_articles, "test.jsonl")

In [ ]:
# Load them as datasets
data_files = {"train": "train.jsonl", "validation": "valid.jsonl", "test": "test.jsonl"}
dataset = load_dataset("json", data_files=data_files)

In [ ]:
dataset

In [ ]:
dataset['train'][0]

# Upload Dataset to Hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
dataset

In [ ]:
dataset.push_to_hub("reuters_articles")